<a href="https://colab.research.google.com/github/yintrigue/melanoma/blob/master/CNN_Trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
from google.colab import files
from google.colab import drive
import os

# optional: mount Google Drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [75]:
%cd /content/gdrive/My Drive/w207
!ls

/content/gdrive/My Drive/w207
melanoma


In [ ]:
!pip install python-resize-image

In [77]:
import pandas as pd
import numpy as np
from pylab import *
from PIL import Image
from resizeimage import resizeimage
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils

In [78]:
train_csv=pd.read_csv('/content/gdrive/My Drive/Kaggle/melanoma/csv/train.csv')
#test_csv=pd.read_csv('/content/gdrive/My Drive/Kaggle/melanoma/csv/test.csv')

In [79]:
test_csv

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,male,70.0,NaN
1,ISIC_0052349,IP_7782715,male,40.0,lower extremity
2,ISIC_0058510,IP_7960270,female,55.0,torso
3,ISIC_0073313,IP_6375035,female,50.0,torso
4,ISIC_0073502,IP_0589375,female,45.0,lower extremity
...,...,...,...,...,...
10977,ISIC_9992485,IP_4152479,male,40.0,torso
10978,ISIC_9996992,IP_4890115,male,35.0,torso
10979,ISIC_9997917,IP_2852390,male,25.0,upper extremity
10980,ISIC_9998234,IP_8861963,male,65.0,lower extremity


In [80]:
malignant_df=train_csv[train_csv['benign_malignant']=='malignant'][:500]
benign_df=train_csv[train_csv['benign_malignant']=='benign'][:2000]

malignant_df_test=train_csv[train_csv['benign_malignant']=='malignant'][500:]
benign_df_test=train_csv[train_csv['benign_malignant']=='benign'][2000:4000]

In [81]:
train_labels_mal=malignant_df['benign_malignant'].tolist()
image_names_mal=malignant_df['image_name'].tolist()
train_labels_ben=benign_df['benign_malignant'].tolist()
image_names_ben=benign_df['image_name'].tolist()

train_labels_mal_test=malignant_df_test['benign_malignant'].tolist()
image_names_mal_test=malignant_df_test['image_name'].tolist()
train_labels_ben_test=benign_df_test['benign_malignant'].tolist()
image_names_ben_test=benign_df_test['image_name'].tolist()

In [82]:
image_names_ben[1]

'ISIC_0015719'

In [83]:
train_data=[]
train_labels=[]
num_mal=0
num_benign=0
for image in image_names_mal:
  try:
    im = Image.open("/content/gdrive/My Drive/Kaggle/melanoma/jpeg_compressed/q_60/train/{}.jpg".format(image))
    im=array(resizeimage.resize('thumbnail', img, [32, 32]))
    train_data.append(im)
    train_labels.append(1)
    num_mal+=1
  except:
    pass

for image in image_names_ben:
    try:
      im = Image.open("/content/gdrive/My Drive/Kaggle/melanoma/jpeg_compressed/q_60/train/{}.jpg".format(image))
      im=array(resizeimage.resize('thumbnail', img, [32, 32]))
      train_data.append(im)
      train_labels.append(0)
      num_benign+=1
    except:
      pass

In [84]:
test_data=[]
test_labels=[]
num_mal=0
num_benign=0
for image in image_names_mal_test:
  try:
    im = Image.open("/content/gdrive/My Drive/Kaggle/melanoma/jpeg_compressed/q_60/train/{}.jpg".format(image))
    im=array(resizeimage.resize('thumbnail', img, [32, 32]))
    test_data.append(im)
    test_labels.append(1)
    num_mal+=1
  except:
    pass

for image in image_names_ben_test:
    try:
      im = Image.open("/content/gdrive/My Drive/Kaggle/melanoma/jpeg_compressed/q_60/train/{}.jpg".format(image))
      im=array(resizeimage.resize('thumbnail', img, [32, 32]))
      test_data.append(im)
      test_labels.append(0)
      num_benign+=1
    except:
      pass

In [85]:
train_data=np.array(train_data)
test_data=np.array(test_data)
X_train = train_data.reshape(train_data.shape[0], 32, 32, 3)
X_test = test_data.reshape(test_data.shape[0], 32, 32, 3)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

Y_train = np_utils.to_categorical(train_labels, 2)
Y_test = np_utils.to_categorical(test_labels, 2)

In [86]:
# building a linear stack of layers with the sequential model
model = Sequential()

# convolutional layer
model.add(Conv2D(50, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=(32, 32, 3)))

# convolutional layer
model.add(Conv2D(75, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(125, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# flatten output of conv
model.add(Flatten())

# hidden layer
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(2, activation='softmax'))

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# training the model for 10 epochs
model.fit(X_train, Y_train, batch_size=128, epochs=2, validation_data=(X_test, Y_test)) #need to update this to use test data


Train on 1223 samples, validate on 1003 samples
Epoch 1/2
1223/1223 [==============================] - 17s 14ms/step - loss: 0.6072 - accuracy: 0.7547 - val_loss: 0.3693 - val_accuracy: 0.9561
Epoch 2/2
1223/1223 [==============================] - 17s 14ms/step - loss: 0.4909 - accuracy: 0.8087 - val_loss: 0.3863 - val_accuracy: 0.9561


In [87]:
model.predict(X_test)[0]

array([0.7061839, 0.2938161], dtype=float32)